In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
# import missingno as msno
import pickle
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  Lasso
import sys
import joblib

sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS



In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive #comment out this line if you are using Colab
drive.mount("/content/gdrive/") 

Mounted at /content/gdrive/


In [ ]:
# importing the csv into DataFrame using Pandas
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Intro to DS//Group_19_data_cleaned.csv",encoding= 'unicode_escape',parse_dates=[0])
df.head(2)

,Unnamed: 0,Date,Rented Bike Count,Hour,Temperature(Â°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(Â°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,0,2017-01-12,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,0,1
1,1,2017-01-12,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,0,1


In [ ]:
df = df.drop("Unnamed: 0",axis=1)

In [ ]:
df.columns = [i.split("(")[0].strip().title().replace(" ","_") for i in [*df.columns]]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   8760 non-null   object 
 1   Rented_Bike_Count      8760 non-null   int64  
 2   Hour                   8760 non-null   int64  
 3   Temperature            8760 non-null   float64
 4   Humidity               8760 non-null   int64  
 5   Wind_Speed             8760 non-null   float64
 6   Visibility             8760 non-null   int64  
 7   Dew_Point_Temperature  8760 non-null   float64
 8   Solar_Radiation        8760 non-null   float64
 9   Rainfall               8760 non-null   float64
 10  Snowfall               8760 non-null   float64
 11  Seasons                8760 non-null   object 
 12  Holiday                8760 non-null   int64  
 13  Functioning_Day        8760 non-null   int64  
dtypes: float64(6), int64(6), object(2)
memory usage: 958.2+ 

In [ ]:
df.Date = pd.to_datetime(df.Date, format="%Y/%m/%d")

In [ ]:
df.insert(1,"Day", df.Date.dt.day)
df.insert(2, "Month", df.Date.dt.month)
df.insert(3, "Year", df.Date.dt.year)

In [ ]:
df.insert(3, 'WeekDay',df["Date"].dt.day_name())

In [ ]:
df.Year = df.Year.map({2017:0, 2018:1})

In [ ]:
df.Seasons = df.Seasons.map({"Winter":1, 'Autumn':2, 'Spring':3, 'Summer':4})

In [ ]:
df.insert(2, 'label_day_night', df['Hour'].apply(lambda x : 0 if (x<7) else( 1)))

In [ ]:
df["Working_Day"] = 1
df.loc[(df.WeekDay=="Saturday")|(df.WeekDay=="Sunday"), "Working_Day"] = 0

In [ ]:
df.Humidity[df.Humidity==0] = df.Humidity.mean()

In [ ]:
categoryVariableList = ["WeekDay"]
for var in categoryVariableList:
    df[var] = df[var].astype("category")

In [ ]:
for col in categoryVariableList:
    df_dummies_col = pd.get_dummies(df[col])
    
    df = pd.concat([df, df_dummies_col],axis=1)
    
    df.drop(col, axis=1, inplace=True)

In [ ]:
df.columns

Index(['Date', 'Day', 'label_day_night', 'Month', 'Year', 'Rented_Bike_Count',
       'Hour', 'Temperature', 'Humidity', 'Wind_Speed', 'Visibility',
       'Dew_Point_Temperature', 'Solar_Radiation', 'Rainfall', 'Snowfall',
       'Seasons', 'Holiday', 'Functioning_Day', 'Working_Day', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [ ]:
columns = df.columns.tolist()
columns.append(columns.pop(5))
df = df[columns]

In [ ]:
df.drop("Date",axis=1, inplace=True)

In [ ]:
numerical_feature = ["Temperature","Humidity","Rainfall",
                     "Visibility","Dew_Point_Temperature",
                     'Wind_Speed','Solar_Radiation',"Snowfall"]

# scale = StandardScaler()
# df[numerical_feature] = scale.fit_transform(df[numerical_feature])

In [ ]:
df

,Day,label_day_night,Month,Year,Hour,Temperature,Humidity,Wind_Speed,Visibility,Dew_Point_Temperature,...,Functioning_Day,Working_Day,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Rented_Bike_Count
0,12,0,1,0,0,-5.2,37.0,2.2,2000,-17.6,...,1,1,0,0,0,0,1,0,0,254
1,12,0,1,0,1,-5.5,38.0,0.8,2000,-17.6,...,1,1,0,0,0,0,1,0,0,204
2,12,0,1,0,2,-6.0,39.0,1.0,2000,-17.7,...,1,1,0,0,0,0,1,0,0,173
3,12,0,1,0,3,-6.2,40.0,0.9,2000,-17.6,...,1,1,0,0,0,0,1,0,0,107
4,12,0,1,0,4,-6.0,36.0,2.3,2000,-18.6,...,1,1,0,0,0,0,1,0,0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30,1,11,1,19,4.2,34.0,2.6,1894,-10.3,...,1,1,1,0,0,0,0,0,0,1003
8756,30,1,11,1,20,3.4,37.0,2.3,2000,-9.9,...,1,1,1,0,0,0,0,0,0,764
8757,30,1,11,1,21,2.6,39.0,0.3,1968,-9.9,...,1,1,1,0,0,0,0,0,0,694
8758,30,1,11,1,22,2.1,41.0,1.0,1859,-9.8,...,1,1,1,0,0,0,0,0,0,712


In [ ]:
X = df.drop(['Rented_Bike_Count'],axis=1)
y = df['Rented_Bike_Count']

In [ ]:
X.columns

Index(['Day', 'label_day_night', 'Month', 'Year', 'Hour', 'Temperature',
       'Humidity', 'Wind_Speed', 'Visibility', 'Dew_Point_Temperature',
       'Solar_Radiation', 'Rainfall', 'Snowfall', 'Seasons', 'Holiday',
       'Functioning_Day', 'Working_Day', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2,random_state =2)

In [ ]:
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
ss=StandardScaler()
X_trains=ss.fit_transform(X_train)
X_tests=ss.fit_transform(X_test)

In [ ]:
# from sklearn.svm import SVR
# svr=SVR()
# svr.fit(X_trains,y_train)

In [ ]:
# forward_fs_best=sfs(estimator = svr, k_features = 'best', forward = True,verbose = 1, scoring = 'r2')
# sfs_forward_best=forward_fs_best.fit(X_trains,y_train)

In [ ]:
# print('R-Squared value:', sfs_forward_best.k_score_)

In [ ]:
# print('Features after Forward Feature Selection using SVM are : ',sfs_forward_best.k_feature_names_)

In [ ]:
from sklearn.svm import SVR
svr1=SVR()
svr1.fit(X_train,y_train)

SVR()

In [ ]:
forward_fs_best1=sfs(estimator = svr1, k_features = 'best', forward = True,verbose = 1, scoring = 'r2')
sfs_forward_best1=forward_fs_best1.fit(X_train,y_train)

In [ ]:
sfs_forward_best1.k_feature_names_

('Hour', 'Temperature')

In [ ]:
print('R-Squared value:', sfs_forward_best1.k_score_)

R-Squared value: 0.4196685472003046


In [ ]:
X_trains_new=X_train[['Hour', 'Temperature']]
sfs_svm=SVR(kernel='rbf',gamma='scale', C=60.0, epsilon=1)
sfs_fit=sfs_svm.fit(X_trains_new, y_train)

In [ ]:

svr_sfs_score = sfs_fit.score(X_test[['Hour', 'Temperature']],y_test)
print('The score on test set with SFS for SVM WITHOUT linear kernel is : {:.2f}%'.format(svr_sfs_score) )

The score on test set with SFS for SVM WITHOUT linear kernel is : 0.52%


In [3]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
print('MAE for test set with hyperparameter tuning for SVM WITHOUT linear kernel with BIDIRECTIONAL variable selection is : {:.2f}'.format((mean_absolute_error(y_test,sfs_fit.predict(X_test[['Hour', 'Temperature']])))))
print('RMSE for test set with hyperparameter tuning for SVM WITHOUT linear kernel with BIDIRECTIONAL variable selection is : {:.2f}'.format(np.sqrt(mean_squared_error(y_test,sfs_fit.predict(X_test[['Hour', 'Temperature']])))))

In [ ]:
Group_19_Bi_directional_elimination_filename = '/content/gdrive/My Drive/Colab Notebooks/Intro to DS/models/Group_19_Bi_directional_elimination.sav'
pickle.dump(sfs_fit, open(Group_19_Bi_directional_elimination_filename, 'wb'))